In [1]:
import numpy as np
import pandas as pd
import warnings
from time import sleep
from tqdm.notebook import tqdm

warnings.filterwarnings("ignore")

from cyber_sdk.key.mnemonic import MnemonicKey
from cyber_sdk.client.lcd.api.tx import CreateTxOptions
from cyber_sdk.core.fee import Fee
from cyber_sdk.core import Coin, Coins, AccAddress
from cyber_sdk.exceptions import LCDResponseError
from cyber_sdk.core.wasm import MsgExecuteContract

from config import BOSTROM_LCD_CLIENT, WALLET_SEED, ipfs_client

In [2]:
mk = MnemonicKey(mnemonic=WALLET_SEED)
WALLET = BOSTROM_LCD_CLIENT.wallet(mk)
WALLET_ADDRESS = WALLET.key.acc_address
SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1823qj9q6eruxv8mfwfay87zd9pp66ayq0ckp9xttk0e758a6kl9qtglw7z'
BASE_COIN_DENOM = 'boot'

## Get data for types of particles

In [3]:
content_type_df = pd.read_csv('data/particles_with_filetype_20230418.csv')[['particle', 'type']]
content_type_to_cid_dict = {_content_type: ipfs_client.add_str(_content_type) for _content_type in list(content_type_df.type.unique())}
content_type_df['type_cid'] = content_type_df['type'].map(lambda content_type: content_type_to_cid_dict[content_type])
content_type_df.head()

,particle,type,type_cid
0,QmfNfxY5AuE5rrDnE8w8TH7n9fPJQwLhvCo8xHELrP657i,audio,QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX
1,QmV4wJWemDkVLcsXxpgwPvMnNNdUdV1NzN3ZgwaUuVbStM,audio,QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX
2,QmdbvwcPfckUzFtcsnvBj28pMua5feXg5k6W1nn5nDeeVT,audio,QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX
3,QmScFB5z8Z9ijWR9TkRJXB7NSHUSsR21z1MURbYMie9ZDr,audio,QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX
4,QmYmbqTjkmfZHDDt15X8WW1xwPmx5eKDjCz5w4k9uF7d6C,audio,QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX


# Create cyberlinks by the subgraph contract

In [4]:
txs_broadcasted = []
for content_type_item_df in tqdm(np.array_split(content_type_df.iloc[9:100], indices_or_sections=9)):
    links = []
    for _, row in content_type_item_df.iterrows():
        links.append({"from": row.particle, "to": row.type_cid})

    execute_msg = {"cyberlink": {"links": links}}

    create_links_msgs = \
        [MsgExecuteContract(
            sender=AccAddress(WALLET_ADDRESS),
            contract=AccAddress(SUBGRAPH_CONTRACT_ADDRESS),
            execute_msg=execute_msg)]

    tx_signed = WALLET.create_and_sign_tx(
        CreateTxOptions(
            msgs=create_links_msgs,
            memo='upload content types github.com/Snedashkovsky/knowledge_graph_research',
            fee=Fee(5_000_000, Coins([Coin(amount=0, denom=BASE_COIN_DENOM)]))
        ))

    print('\n'.join([f"{link['from']} -> {link['to']}" for link in links]))
    try:
        tx_broadcasted = BOSTROM_LCD_CLIENT.tx.broadcast(tx_signed)
        txs_broadcasted.append(tx_broadcasted)
    except LCDResponseError as e:
        print(f'LCDResponseError: {e}')
    sleep(10)

  0%|          | 0/9 [00:00<?, ?it/s]

QmcJFpwHofQjtSrnpxjekCSjZmnHZhGK2KR7eeeBEKjKAZ -> QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX
QmY6dsuGeLLQMdseCgWCh4Uu3keK5iXXfo8wfUUWM5SRMR -> QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX
QmbYwLb7gxZumzFfMJTgBaThuZ9Bs247EhVTAZ4srAhe5b -> QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX
QmVSb9zX5aiHwQDCGoCpmAgXhG8qTJ28zwKtqwauMJRPLh -> QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX
QmSWUxCEPwmqkX2xYtGq355DD4pYSSsVvxVddqUidAiGXj -> QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX
QmVQJbYu4gKsn52iebHFfqdnnQCDpW2o4mz4tTXk4fCgXG -> QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX
QmTwiEczgjysEvJ1qu1fzBhoj1vmhwjNWCQmxPqpFPpfBr -> QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX
QmTw5jkVwDJTs6rswNeitKRMyzY22BxBGdeKncu4eDy3vL -> QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX
QmfNx8p4wsMVAdP5AWESCji6hBtNyEthFWw3zmSUtnMQUh -> QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX
Qmc1wWZu73HuKXTjsEdDQ5nZKTW4Xishe6F5p9nYo9dvQm -> QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX
QmPnJERZ5yumFfRnLjp3wPzdvyvNL1